In [1]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [ ]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
            driver.quit() # Close the driver
        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('assam_bus_details.csv', index=False)





Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/goalpara-to-guwahati: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6DC0D6CF5+28821]
	(No symbol) [0x00007FF6DC043880]
	(No symbol) [0x00007FF6DBEE578A]
	(No symbol) [0x00007FF6DBF391BE]
	(No symbol) [0x00007FF6DBF394AC]
	(No symbol) [0x00007FF6DBF82647]
	(No symbol) [0x00007FF6DBF5F33F]
	(No symbol) [0x00007FF6DBF7F412]
	(No symbol) [0x00007FF6DBF5F0A3]
	(No symbol) [0x00007FF6DBF2A778]
	(No symbol) [0x00007FF6DBF2B8E1]
	GetHandleVerifier [0x00007FF6DC40FCED+3408013]
	GetHandleVerifier [0x00007FF6DC42745F+3504127]
	GetHandleVerifier [0x00007FF6DC41B63D+3455453]
	GetHandleVerifier [0x00007FF6DC19BDFB+835995]
	(No symbol) [0x00007FF6DC04EB9F]
	(No symbol) [0x00007FF6DC04A854]
	(No symbol) [0x00007FF6DC04A9ED]
	(No symbol) [0x00007FF6DC03A1D9]
	BaseThreadInitThunk [0x00007FFEAC89259D+29]
	RtlUserThreadStart [0x00007FFEAD26AF38+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-t

In [2]:
import pandas as pd

# Read the CSV file with the absolute path
df = pd.read_csv(r"C:\Users\abina\OneDrive\Desktop\visualcode\redbusScraping\exportedcsvfiles\assam_bus_details.csv")

# Print the first few rows to check the data
print(df.head())

           Route_Name                                         Route_Link  \
0  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
1  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
2  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
3  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   
4  Tezpur to Guwahati  https://www.redbus.in/bus-tickets/tezpur-to-gu...   

                                            Bus_Name  \
0  Assam State Transport Corporation (ASTC) - 156590   
1  Assam State Transport Corporation (ASTC) - 139847   
2                               Chartered Bus - ASTC   
3                       Pranjit Travels (Under ASTC)   
4                                      Laxmi Travels   

                       Bus_Type Departing_Time Duration Reaching_Time  \
0  Bharat Benz A/C Seater (2+2)          14:00  04h 30m         18:30   
1           Volvo AC Seater 2+2          16:15  04h 15m     

In [3]:

df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 156590,Bharat Benz A/C Seater (2+2),14:00,04h 30m,18:30,3.9,INR 298,12 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Assam State Transport Corporation (ASTC) - 139847,Volvo AC Seater 2+2,16:15,04h 15m,20:30,4.2,INR 298,14 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Chartered Bus - ASTC,A/C Seater Push Back (2+2),11:30,04h 45m,16:15,4.4,INR 397.08,15 Seats available
3,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Pranjit Travels (Under ASTC),NON A/C Seater (2+1),11:30,05h 15m,16:45,4.1,INR 450,29 Seats available
4,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Laxmi Travels,A/C Seater (2+1),12:30,04h 45m,17:15,4.2,475,28 Seats available
...,...,...,...,...,...,...,...,...,...,...
251,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Yatra Travels,A/C Seater / Sleeper (2+1),21:00,06h 30m,03:30,3.9,585,36 Seats available
252,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,Yatra Travels,NON A/C Seater (2+1),20:40,06h 20m,03:00,4.1,540,17 Seats available
253,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,PARIJAT TRAVELS(UNDER ASTC),NON A/C Seater (2+1),20:56,06h 04m,03:00,4.4,INR 590,23 Seats available
254,North Lakhimpur to Nagaon (Assam),https://www.redbus.in/bus-tickets/north-lakhim...,DESTINY TRAVELS,NON AC Seater / Sleeper 2+1,18:30,07h 00m,01:30,1.5,INR 498.75,35 Seats available
